In [2]:
import sys
sys.path.append('..')

import torch
import lightning as L
from torch.utils.data import DataLoader

from dataset import AEDataset
from lightning_model.autoencoder import LitAE

In [2]:
from types import SimpleNamespace
config = {
    'enc_filters': (64, 128, 128, 256),
    'latent_dim': 128,
    'enc_bn': True,
    'dec_features': (256, 256),
    'n_pts': 256,
    'dec_bn': False,
}
config = SimpleNamespace(**config)

In [3]:
root = '..'
dataset_name = 'shapenetcorev2'
# choose split type from 'train', 'test', 'all', 'trainval' and 'val'
# only shapenetcorev2 and shapenetpart dataset support 'trainval' and 'val'

train_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='train')
val_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='val')
test_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
print("datasize:", train_dataset.__len__())
print("datasize:", val_dataset.__len__())
print("datasize:", test_dataset.__len__())

datasize: 35708
datasize: 5158
datasize: 10261


In [4]:
autoencoder = LitAE(config)

In [5]:
trainer = L.Trainer(max_epochs=10000, gpus=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=val_loader)

/home/tsa87/anaconda3/envs/imle/lib/python3.7/site-packages/lightning/pytorch/trainer/connectors/accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type    | Params
----------------------------------------
0 | autoencoder | PointAE | 390 K 
----------------------------------------
390 K     Trainable params
0         Non-trainable params
390 K     Total params
1.564     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/tsa87/anaconda3/envs/imle/lib/python3.7/site-packages/lightning/pytorch/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
